In [4]:
'''
- entity type: how many sentences, how many of this type in the corpus
we can do 80/10/10
and for each set we should calculate:
no. of sentences
no. of  tokens
no. of  tokens of each class
entity span length (geometric mean)
span distinctiveness
boundary distinctiveness 
the span distinctiveness is the KL divergence of overall word distribution vs entity word dist
the boundary distinctiveness is the KL divergence of border words (tokens directly before and after entity) dist and overall dist
'''

UsageError: Line magic function `%which` not found.


In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import math
import random

### load in the corpus

In [58]:
nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [64]:
corpus = f'{os.getcwd()}/../json Files/complete_corpus.json'
with open(corpus, 'r') as file:
    content = file.read()
    corpus = json.loads(content)
print(f'{sum([len(c) for c in corpus])} total annotated sentences in corpus')
print(len(corpus))

406 total annotated sentences in corpus
3


In [153]:
corpus_v2 = f'{os.getcwd()}/../json Files/complete_annotations_v2.json'
with open(corpus_v2, 'r') as file:
    content = file.read()
    corpus_v2 = json.loads(content)
print(len(corpus_v2))

396


In [154]:
sentences = set()
duplicate_sentence_ids = set()
for item in corpus_v2:
    if corpus_v2[item]["sentence"]["text"] in sentences:
        duplicate_sentence_ids.add(corpus_v2[item]["id"])
    else:
        sentences.add(sentences.add(corpus_v2[item]["sentence"]["text"]))
print(len(duplicate_sentence_ids))
print(duplicate_sentence_ids)

0
set()


### Splitting the dataset

In [120]:
train_size = math.floor(len(corpus_v2) * 0.80)
while train_size % 2 == 1:
    train_size -= 1
test_size, dev_size = (len(corpus_v2) - train_size) / 2, (len(corpus_v2) - train_size) / 2
print(f'train size: {train_size}\ntest size: {test_size}\n dev size: {dev_size}')
assert train_size + test_size + dev_size == len(corpus_v2)
train_size = int(train_size)
test_size = int(test_size)
dev_size = int(dev_size)

train size: 316
test size: 40.0
 dev size: 40.0


In [128]:
corpus_v2_as_list = list(corpus_v2)
assert len(set(corpus_v2_as_list)) == len(corpus_v2_as_list)
random.shuffle(corpus_v2_as_list)
print(train_size, train_size + test_size )
train_sentences = corpus_v2_as_list[:train_size]
test_sentences = corpus_v2_as_list[train_size:train_size + test_size]
dev_sentences = corpus_v2_as_list[train_size + test_size::]

assert len(train_sentences) + len(test_sentences) + len(dev_sentences) == len(corpus_v2_as_list)

train_json, test_json, dev_json = {}, {}, {}
i = 0
for id in train_sentences:
    train_json[id] = corpus_v2[id]

for id in test_sentences:
    test_json[id] = corpus_v2[id]

for id in dev_sentences:
    dev_json[id] = corpus_v2[id]

assert len(train_json) + len(test_json) + len(dev_json) == len(corpus_v2)

filepath = f'{os.getcwd()}/../json Files/'

train_json = json.dumps(train_json, indent=4)
with open(filepath + "train-split.json", "w") as outfile:
    outfile.write(train_json)

test_json = json.dumps(test_json, indent=4)
with open(filepath + "test-split.json", "w") as outfile:
    outfile.write(test_json)

dev_json = json.dumps(dev_json, indent=4)
with open(filepath + "dev-split.json", "w") as outfile:
    outfile.write(dev_json)

316 356


In [98]:
df = pd.DataFrame(columns=["sentences"])
for i in range(20):
    df.loc[i] = [0]
df.head()

,sentences
0,0
1,0
2,0
3,0
4,0


In [79]:
NER_categories = {}
ids = []
l = []
g = []
e = []
for corpus_part in corpus:
    for item in corpus_part:
        object = corpus_part[item]
        tokens = tokenizer(object["sentence"]["text"])
        NER_categories = object["NER_categories"]
        tag_categories = object["token_tags"]
        if len(NER_categories) != len(tokens):
            ids.append(object["id"])
        elif len(NER_categories) < len(tokens):
            l.append(object["id"]) 
        elif len(NER_categories) > len(tokens):
            g.append(object["id"])
        else:
            e.append(object["id"])

In [80]:
print(len(ids), len(l), len(g), len(e))
print(ids)
print(l)
print(g)

2 0 0 404
['300-0', '300-5']
[]
[]


In [54]:
NER_categories = {}
for corpus_part in corpus:
    for item in corpus_part:
        object = corpus_part[item]
        NER_categories = object["NER_categories"]
        i = 0
        while i < len(NER_categories):
            if NER_categories[i] == "0": 
                i += 1
                continue
            else:
                j = i
                while j < len(NER_categories) and NER_categories[j] == NER_categories[i]:
                    j += 1
                try:
                    df.loc[int(NER_categories[i]), "sentences"] += 1        
                except Exception as e:
                    print(object["id"])
                    print(NER_categories[i])
                i = j

### load in the split json files

{'487-6': {'id': '487-6', 'sentence': {'sentence_id': 6, 'text': 'eSIMs make it easy to connect and manage a large number of IoT devices across different locations, even in remote and harsh environments.'}, 'NER_categories': ['7', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 'token_tags': ['O', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']}, '337-6': {'id': '337-6', 'sentence': {'sentence_id': 6, 'text': 'For instance, cybercriminals were able to turn off the heating in two buildings in the Finnish city of Lappeenranta, and other attacks such as Shamoon, New York Dam, and the Mirai Botnet have affected industrial IoT devices such as Deutsche Telekom Routers, resulting in the disruption of services.'}, 'NER_categories': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '11', '0', '0', '0', '0', '0', '11', '0', '0', '0', '0', '0', '6', '18', '18', '

In [159]:
def analyze_token_types_in_split(corpus):
    df = pd.DataFrame(columns=["sentences", "tokens"])
    for i in range(20):
        df.loc[i] = [0, 0]
    df.head()
    NER_categories = {}
    for item in corpus:
        object = corpus[item]
        tokens = tokenizer(object["sentence"]["text"]) # spacy tokenizer
        NER_categories = object["NER_categories"]
        i = 0
        while i < len(NER_categories):
            if NER_categories[i] == "0": 
                i += 1
                df.loc[0, "tokens"] += 1
                df.loc[0, "sentences"] += 1
                continue
            else:
                j = i
                while j < len(NER_categories) and NER_categories[j] == NER_categories[i]:
                    j += 1
                try:
                    df.loc[int(NER_categories[i]), "sentences"] += 1        
                    df.loc[int(NER_categories[i]), "tokens"] += j - i
                except Exception as e:
                    print(object["id"])
                    print(NER_categories[i])
                i = j
    '''
    no. of sentences
    no. of  tokens
    no. of  tokens of each class
    '''
    print(f'Number of sentences in split: {len(corpus)}')
    print(f'Number of tokens {df["tokens"].sum()}')
    print(df)

In [160]:
filepath = f'{os.getcwd()}/../json Files/'
train_data, dev_data, test_data = None, None, None
# Load JSON files
with open(os.path.join(filepath, 'train-split.json'), 'r') as train_file:
    train_data = json.load(train_file)

with open(os.path.join(filepath, 'dev-split.json'), 'r') as dev_file:
    dev_data = json.load(dev_file)

with open(os.path.join(filepath, 'test-split.json'), 'r') as test_file:
    test_data = json.load(test_file)

corpus_v2 = f'{os.getcwd()}/../json Files/complete_annotations_v2.json'
with open(corpus_v2, 'r') as file:
    content = file.read()
    corpus_v2 = json.loads(content)

print('whole corpus')
analyze_token_types_in_split(corpus_v2)
print('----------------')
print('train_data')
analyze_token_types_in_split(train_data)
print('----------------')
print('test_data')
analyze_token_types_in_split(test_data)
print('----------------')
print('dev_data')
analyze_token_types_in_split(dev_data)
print('----------------')


whole corpus
Number of sentences in split: 396
Number of tokens 8068
    sentences  tokens
0        6462    6462
1          78     128
2          84     148
3          17      36
4           6      10
5         143     242
6          22      51
7          81     136
8          38      71
9          44      85
10          1       2
11         59      98
12         56     122
13         17      44
14          8      11
15         41      58
16         54      99
17         49      65
18         81     182
19         10      18
----------------
train_data
Number of sentences in split: 316
Number of tokens 6382
    sentences  tokens
0        5148    5148
1          60     101
2          60      96
3          16      34
4           5       9
5         113     193
6          16      39
7          59      96
8          29      51
9          41      80
10          1       2
11         54      91
12         42      86
13          7      19
14          8      11
15         35      48
16         

In [167]:
def lists_for_ent_span(corpus):
    output = {}
    NER_categories = {}
    for item in corpus:
        object = corpus[item]
        tokens = tokenizer(object["sentence"]["text"]) # spacy tokenizer
        NER_categories = object["NER_categories"]
        ents = []
        all = []
        surrounding_ents = []
        for token in tokens:
            all.append(token.text)
        #print(f'all: {all}')

        i = 0
        while i < len(NER_categories):
            if NER_categories[i] == "0":
                surrounding_ents.append(tokens[i].text)
                i += 1
                continue
            else:
                j = i
                while j < len(NER_categories) and NER_categories[j] == NER_categories[i]:
                    j += 1
                for ent in tokens[i:j]:
                    ents.append(ent.text)
                i = j
                
        #print(f'surrounding: {surrounding_ents}')
        #print(f'ents {ents}')
        output[object["id"]] = {
            "all-tokens": all,
            "entities": ents,
            "surrounding-tokens":surrounding_ents,
        }
    assert len(output) == len(corpus)
    return output

entity_span_train = lists_for_ent_span(train_data)
entity_span_test = lists_for_ent_span(test_data)
entity_span_dev = lists_for_ent_span(dev_data)


def save_to_json(data, filename):
    with open(f'{os.getcwd()}/../json Files/{filename}', 'w') as json_file:
        json.dump(data, json_file, indent=4)

save_to_json(entity_span_train, 'entity_span_train.json')
save_to_json(entity_span_test, 'entity_span_test.json')
save_to_json(entity_span_dev, 'entity_span_dev.json')

print("Entity spans have been saved to JSON files.")


Entity spans have been saved to JSON files.
